# Enfoque econométrico

Para el enfoque econométrico usamos múltples modelos:

- OLS
- Fixed Effect
- Random Effect

Se van a probar ambos con la prueba Hausman, y la descomposición de varianza del mejor modelo.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels import PanelOLS, RandomEffects
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Load Raw Data
df_raw = pd.read_excel("../data/raw/Viajes Sep-Dic 24 v2.xlsx", sheet_name='Viajes')
df_raw = df_raw[df_raw['Viaje'].notna()].copy()

print(f"Loaded {len(df_raw)} trips")
print(f"Date range: {df_raw['F.Salida'].min()} to {df_raw['F.Salida'].max()}")

## Cargar y limpiar datos

In [ ]:
# Basic Cleaning and Time Features
df_raw['F.Salida'] = pd.to_datetime(df_raw['F.Salida'], errors='coerce')
df_raw['Mes'] = df_raw['F.Salida'].dt.month
df_raw['DiaSemana'] = df_raw['F.Salida'].dt.dayofweek
df_raw['Semana'] = df_raw['F.Salida'].dt.isocalendar().week

# Route statistics (proxy for distance/complexity)
route_stats = df_raw.groupby('Ori-Dest').agg({
    'Costo': ['mean', 'std', 'count'],
    'Peso Total (kg)': 'mean'
}).reset_index()
route_stats.columns = ['Ori-Dest', 'Costo_Mean_Route', 'Costo_Std_Route', 'Route_Freq', 'Peso_Mean_Route']

df = pd.merge(df_raw, route_stats, on='Ori-Dest', how='left')

# Log transforms
df['log_Costo'] = np.log(df['Costo'] + 1)
df['log_Peso'] = np.log(df['Peso Total (kg)'] + 1)

# Categorical features
df['Es_Express'] = (df['TpoSrv'] == 'EX').astype(int)
df['Es_Revestidos'] = (df['Tipo planta'] == 'Revestidos').astype(int)
df['Es_Masivos'] = (df['Tipo planta'] == 'Masivos').astype(int)

# IDs
df['Ruta_ID'] = df['Ori-Dest']
df['Carrier_ID'] = df['Transp.Leg']
df['Carrier_Name'] = df['Nombre']

print("Feature engineering completo")
print(f"Unique routes: {df['Ruta_ID'].nunique()}")
print(f"Unique carriers: {df['Carrier_ID'].nunique()}")

### Estadísticas por transportista

Ayudarán más adelante a identificar cuales transportistas verdaderamente cobran más por los mismos servicios.

In [ ]:
# Carrier Statistics
carrier_stats = df.groupby('Carrier_ID').agg({
    'Costo': ['mean', 'std', 'count'],
    'Nombre': 'first'
}).reset_index()
carrier_stats.columns = ['Carrier_ID', 'Carrier_Cost_Mean', 'Carrier_Cost_Std', 'Carrier_Trips', 'Carrier_Name']

df = pd.merge(df, carrier_stats, on='Carrier_ID', how='left')

print("Top 10 transportistas:")
print(carrier_stats.nlargest(10, 'Carrier_Trips')[['Carrier_Name', 'Carrier_Trips', 'Carrier_Cost_Mean']])

## Dataset final

In [ ]:
# Clean Dataset
df_clean = df.dropna(subset=['Costo', 'Peso Total (kg)', 'Carrier_ID', 'Ruta_ID']).copy()

print(f"Clean dataset: {len(df_clean)} trips ({100*len(df_clean)/len(df):.1f}% retained)")

# Create dummy variables
top_carriers = df_clean['Carrier_ID'].value_counts().head(10).index
df_clean['Carrier_Group'] = df_clean['Carrier_ID'].apply(
    lambda x: x if x in top_carriers else 'Other'
)
carrier_dummies = pd.get_dummies(df_clean['Carrier_Group'], prefix='Carrier', drop_first=True)

top_routes = df_clean['Ruta_ID'].value_counts().head(20).index
df_clean['Route_Group'] = df_clean['Ruta_ID'].apply(
    lambda x: x if x in top_routes else 'Other'
)
route_dummies = pd.get_dummies(df_clean['Route_Group'], prefix='Route', drop_first=True)

print(f"Created {len(carrier_dummies.columns)} carrier dummies")
print(f"Created {len(route_dummies.columns)} route dummies")

# Modelo OLS

Antes de hacer los modelos Fixed Effect y Random Effect, quiero comenzar con OLS, que está más limitado pero llega bastante lejos con las variables correctas. Este me sirve como baseline para los otros modelos.
Este modelo controla por el peso, las rutas, si el pedido es express, si es revestido (o masivo), los transportistas y las rutas.

Antes de el modelo con todas las variables, hago modelos con menos variables para observar la diferencia entre sus $R^2$

In [ ]:
y = df_clean['Costo'].values
X_base = df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']].values
X_base = sm.add_constant(X_base)

model_pooled = sm.OLS(y, X_base).fit()

print(f"\nR^2 solo peso y servicio: {model_pooled.rsquared:.3f}")

X_route_data = pd.concat([
    df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']],
    route_dummies
], axis=1)

X_route = X_route_data.astype(float).values
X_route = sm.add_constant(X_route)

model_route = sm.OLS(y, X_route).fit()

print(f"\nR^2 peso, servicio y ruta: {model_route.rsquared:.3f}")
print(f"Adjusted R-squared: {model_route.rsquared_adj:.3f}")

print(f"\nDiferencia de R^2 con solo servicio: {100*(model_route.rsquared - model_pooled.rsquared):.1f}%")

In [ ]:
X_full_data = pd.concat([
    df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']],
    route_dummies,
    carrier_dummies
], axis=1)

X_full = X_full_data.astype(float).values
X_full = sm.add_constant(X_full)

model_full = sm.OLS(y, X_full).fit()

print(f"\nR^2 completa: {model_full.rsquared:.3f}")
print(f"R^2 ajustada: {model_full.rsquared_adj:.3f}")
print(f"\nKey Coefficients:")
print(f"Weight: ${model_full.params[1]:.2f} per kg (p={model_full.pvalues[1]:.4f})")
print(f"Express: ${model_full.params[2]:.2f} (p={model_full.pvalues[2]:.4f})")
print(f"Revestidos: ${model_full.params[3]:.2f} (p={model_full.pvalues[3]:.4f})")

En esta ejecución del modelo no muestro el efecto de los dummies de los transportistas, porque son muchos, pero debajo pongo una mejor manera de visualizar el efecto.

### Efecto de los transportistas:

In [ ]:
carrier_effects = []
carrier_names_map = dict(zip(carrier_stats['Carrier_ID'], carrier_stats['Carrier_Name'])) # carrier_names_map[103079] da 'TRANSPORTES ORTA S.A DE C.V'

# The carrier coefficients start after: const + 3 base vars + route dummies
n_base = 4
n_routes = len(route_dummies.columns)
carrier_coef_start = n_base + n_routes

for i, col in enumerate(carrier_dummies.columns):
    coef_idx = carrier_coef_start + i
    carrier_id = col.replace('Carrier_', '')
    # El dummy es un float por alguna razón, hay que corregir eso para mapear al transportista con su id
    try:
        carrier_id_int = int(float(carrier_id))
        carrier_name = carrier_names_map.get(carrier_id_int, 'Unknown')
    except ValueError:
        carrier_name = 'Unknown'  # Handle cases where conversion fails
        
    effect = model_full.params[coef_idx]
    pval = model_full.pvalues[coef_idx]
    
    carrier_effects.append({
        'Carrier_ID': carrier_id,
        'Carrier_Name': carrier_name,
        'Premium': effect,
        'P_value': pval,
        'Significant': 'Yes' if pval < 0.05 else 'No'
    })

carrier_effects_df = pd.DataFrame(carrier_effects).sort_values('Premium', ascending=False)
print(carrier_effects_df.to_string(index=False))

El efecto de cada transportista ahora se puede ordenar observando los 10 transportistas mas grandes entre los datos. Resalta especialemente que **TRANSPORTES ORTA** es el transportista más grande para
Ternium con mas de 1700 viajes, pero carga alrededor de 2800$ más por le mismo viaje, en promedio. Eso se convierte en una diferencia de 4,900,000 entre todos los viajes que hace.

### Descomposición de la varianza (OLS)

También como un tipo de baseline.

In [ ]:
total_var = df_clean['Costo'].var()

# Between-route variance
route_means = df_clean.groupby('Ruta_ID')['Costo'].mean()
between_route_var = route_means.var()

# Between-carrier variance
carrier_means = df_clean.groupby('Carrier_ID')['Costo'].mean()
between_carrier_var = carrier_means.var()

# Explained variance from models
explained_by_weight_service = model_pooled.rsquared * total_var
explained_by_routes = (model_route.rsquared - model_pooled.rsquared) * total_var
explained_by_carriers = (model_full.rsquared - model_route.rsquared) * total_var
residual_var = (1 - model_full.rsquared) * total_var

print(f"Total Variance:          ${total_var:,.0f}")
print(f"\nDecomposition:")
print(f"Weight + Service Type:   ${explained_by_weight_service:,.0f} ({100*model_pooled.rsquared:.1f}%)")
print(f"Route Choice:            ${explained_by_routes:,.0f} ({100*(model_route.rsquared - model_pooled.rsquared):.1f}%)")
print(f"Carrier Choice:          ${explained_by_carriers:,.0f} ({100*(model_full.rsquared - model_route.rsquared):.1f}%)")
print(f"Unexplained:             ${residual_var:,.0f} ({100*(1-model_full.rsquared):.1f}%)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 1. Carrier Premiums
ax1 = axes[0]
carrier_sig = carrier_effects_df[carrier_effects_df['Significant'] == 'Yes'].head(10)
if len(carrier_sig) > 0:
    ax1.barh(carrier_sig['Carrier_Name'], carrier_sig['Premium'])
    ax1.axvline(0, color='red', linestyle='--', linewidth=1)
    ax1.set_xlabel('Cost Premium (MXN)')
    ax1.set_title('Carrier Cost Premium (Top 10, Significant Only)', fontweight='bold')
    ax1.invert_yaxis()

# 2. Model R-squared Comparison
ax2 = axes[1]
r2_values = [model_pooled.rsquared, model_route.rsquared, model_full.rsquared]
model_names = ['Baseline', 'Route Controls', 'Route + Carrier']
bars = ax2.bar(model_names, r2_values)
ax2.set_ylim(0, 1)
ax2.set_ylabel('R-squared')
ax2.set_title('Model Performance Comparison', fontweight='bold')
for i, v in enumerate(r2_values):
    ax2.text(i, v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## Conclusiones OLS

Según nuestro modelo OLS, la ruta determina la mayoría del costo, seguido del peso y tipo de servicio, y finalmente los transportistas. El resto de la varianza no se explica en este modelo, pero prefiero no sobreajustar el modelo base.

Lo que consideramos que propone este modelo es que la ruta misma ya explica gran parte de los costos en tiempo, combustible, etc., y sonbre este costo es que cada transportista pone encima su utilidad. 

## Validacion Rigurosa

### 1. Validación Cruzada Temporal (No Aleatoria) 📅
Para evaluar la capacidad predictiva y la generalización de nuestro modelo a períodos futuros, se utilizó una metodología de Validación Cruzada Temporal de Ventana Expandida (Expanding-Window). Esta técnica es esencial en el análisis de series de tiempo, ya que respeta la estructura cronológica de los datos y simula la predicción en un entorno real.

Metodología:

Split 1: Entrenamiento con datos de Septiembre y Octubre; Prueba con los viajes de Noviembre.

Split 2: Entrenamiento con Septiembre, Octubre y Noviembre; Prueba con los viajes de Diciembre.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


X_full_data = pd.concat([
    df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']],
    route_dummies,
    carrier_dummies
], axis=1).astype(float)
X_full_data = sm.add_constant(X_full_data, has_constant='add')
X_full_data['Mes'] = df_clean['F.Salida'].dt.month 

# ======================================================================
# *** 2. FUNCIÓN DE VALIDACIÓN TEMPORAL (VENTANA EXPANDIDA) ***
# ======================================================================

def temporal_cross_validation(X_data, y_data, train_months, test_months):
    """
    Realiza un split temporal no aleatorio y evalúa el modelo OLS (o el elegido).
    """
    
    train_mask = X_data['Mes'].isin(train_months)
    test_mask = X_data['Mes'].isin(test_months)

    X_train = X_data[train_mask].drop(columns=['Mes'])
    X_test = X_data[test_mask].drop(columns=['Mes'])
    y_train = y_data[train_mask]
    y_test = y_data[test_mask]

    if X_train.empty or X_test.empty:
        return None, None

    model = sm.OLS(y_train, X_train).fit() 
    y_pred = model.predict(X_test)

    # Cálculo de R^2 Out-of-Sample
    ss_total = np.sum((y_test - np.mean(y_test))**2)
    ss_residual = np.sum((y_test - y_pred)**2)
    r_squared_test = 1 - (ss_residual / ss_total)
    rmse = np.sqrt(np.mean((y_test - y_pred)**2))

    return rmse, r_squared_test

results = []
rmse_1, r2_1 = temporal_cross_validation(X_full_data, y, [9, 10], [11])
if rmse_1 is not None:
    results.append({'Train Period': 'Sep-Oct', 'Test Period': 'Nov', 'RMSE': rmse_1, 'R-squared (Test)': r2_1})

rmse_2, r2_2 = temporal_cross_validation(X_full_data, y, [9, 10, 11], [12])
if rmse_2 is not None:
    results.append({'Train Period': 'Sep-Nov', 'Test Period': 'Dic', 'RMSE': rmse_2, 'R-squared (Test)': r2_2})

validation_df = pd.DataFrame(results)

# Benchmarking In-Sample
X_full_no_month = X_full_data.drop(columns=['Mes'])
model_full_fit = sm.OLS(y, X_full_no_month).fit()
r_squared_full_train = model_full_fit.rsquared

print("\n--- Resultados de Validación Cruzada Temporal ---")
print(validation_df.to_string(index=False, float_format='%.4f'))
print(f"\nR-squared (In-Sample, Full Data): {r_squared_full_train:.4f}")

Los resultados demuestran que el modelo tiene una capacidad explicativa fuerte, incluso fuera de la muestra, lo cual es un indicador de gran robustez.

Alto Poder Predictivo: El R 
2
  en la muestra (In-Sample R 
2
 ≈0.72) es alto y se mantiene sólido en los períodos de prueba (Out-of-Sample R >0.66). Esto es una señal excelente.

Generalización Consistente: La pequeña disminución en el R 
2
  de Noviembre (0.6989) a Diciembre (0.6662) sugiere una ligera caída en el poder predictivo a medida que nos acercamos al final del periodo (posiblemente debido a la estacionalidad de fin de año o al aumento de la varianza del costo), pero el rendimiento se mantiene fuerte en general.

Conclusión: La validación cruzada temporal confirma que las relaciones estructurales encontradas en el modelo principal son estables y se generalizan bien a datos futuros. Esto refuerza la validez de las inferencias sobre el Premium de los transportistas.

### Test de robustez

Verificar si el signo y la significancia de las variables clave (Peso y Transportistas) se mantienen estables al cambiar la forma funcional del modelo de lineal a log-lineal (log-log).

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Creamos la matriz X usando log_Peso en lugar de Peso Total (kg)
X_log_data = pd.concat([
    df_clean['log_Peso'],  # Variable logarítmica
    df_clean[['Es_Express', 'Es_Revestidos']],
    route_dummies,
    carrier_dummies
], axis=1).astype(float)

X_log = sm.add_constant(X_log_data, has_constant='add')
y_log = df_clean['log_Costo']

model_log = sm.OLS(y_log, X_log).fit()

# --- 3. FIT MODELO LINEAL ORIGINAL (BASE) ---
# Se necesita recrear el modelo lineal base para la comparación directa.
X_linear_data = pd.concat([
    df_clean['Peso Total (kg)'],
    df_clean[['Es_Express', 'Es_Revestidos']],
    route_dummies,
    carrier_dummies
], axis=1).astype(float)
X_linear = sm.add_constant(X_linear_data, has_constant='add')
y_linear = df_clean['Costo']
model_linear = sm.OLS(y_linear, X_linear).fit()

carrier_name = carrier_dummies.columns[0] 

peso_idx = 1 
carrier_idx = 1 + 3 + len(route_dummies.columns) + 0 

results = pd.DataFrame({
    'Modelo': ['Base (Lineal)', 'Robustez (Logarítmico)'],
    'R2 Ajustado': [model_linear.rsquared_adj, model_log.rsquared_adj],
    'Coef. Peso/log_Peso': [model_linear.params[peso_idx], model_log.params[peso_idx]],
    'P-valor Peso': [model_linear.pvalues[peso_idx], model_log.pvalues[peso_idx]],
    f'Coef. {carrier_name}': [model_linear.params[carrier_idx], model_log.params[carrier_idx]],
    f'P-valor {carrier_name}': [model_linear.pvalues[carrier_idx], model_log.pvalues[carrier_idx]]
})

print("\n--- Resultados de Robustez: Modelo Logarítmico vs. Lineal ---")
print(results.to_string(index=False, float_format='%.4f'))

Los resultados demuestran una robustez sólida en las principales inferencias del modelo:

Robustez del Transportista (Hallazgo Clave):

El signo del transportista clave (Carrier_100139.0) es robusto (negativo en ambos modelos, indicando un descuento constante).

La significancia estadística es extremadamente alta (P≤0.0040 en el modelo Lineal y P=0.0000 en el Log-Log).

Implicación: La conclusión sobre el Premium/Descuento de este transportista se mantiene firme, independientemente de si la relación Costo-Peso es lineal o logarítmica.

Mejora del Ajuste del Modelo: El R 
2
  Ajustado aumentó significativamente (de 0.7210 a 0.8630) en la especificación logarítmica. Esto sugiere que la forma funcional log-log es estructuralmente superior para modelar la relación entre el Costo y sus impulsores, probablemente por capturar mejor las economías de escala.

Elasticidad del Peso: El coeficiente del peso perdió significancia en el modelo logarítmico (P=0.1318), indicando que, si bien el costo marginal es positivo (elasticidad de 0.0180), su impacto no es estadísticamente significativo cuando el modelo se ajusta a la forma Log-Log, lo que minimiza su rol frente a la elección del transportista o la ruta.

### 3. Análisis de Estabilidad Temporal

El Test de Chow compara la suma de los errores cuadrados (RSS) del modelo estimado en todo el período (Modelo Restringido) contra la suma de los errores cuadrados de los modelos estimados por separado en dos subperíodos (Modelos No Restringidos).

Punto de Quiebre (t 
∗
 ): Utilizaremos la transición de Octubre a Noviembre (después del 31 de octubre) para dividir la muestra en dos mitades.

Hipótesis Nula (H 
0
​	
 ): Los coeficientes son iguales en ambos períodos (Hay estabilidad).

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import f

# Creamos la matriz X completa
X_full_data = pd.concat([
    df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']],
    route_dummies,
    carrier_dummies
], axis=1).astype(float)
X = sm.add_constant(X_full_data, has_constant='add')
k = X.shape[1] # Número de coeficientes (incluye la constante)
N_total = len(df_clean)

# --- 2. MODELO RESTRINGIDO (R) - Full Data ---
model_R = sm.OLS(y, X).fit()
RSS_R = model_R.ssr

# Punto de Quiebre: Después del 31 de Octubre
split_date = pd.to_datetime('2024-10-31') 

# Período 1: Septiembre y Octubre
mask_1 = df_clean['F.Salida'] <= split_date
X1 = X[mask_1]
y1 = y[mask_1]
model_1 = sm.OLS(y1, X1).fit()
RSS_1 = model_1.ssr
N1 = len(y1)

# Período 2: Noviembre y Diciembre
mask_2 = df_clean['F.Salida'] > split_date
X2 = X[mask_2]
y2 = y[mask_2]
N2 = len(y2)

# Comprobación de suficiencia de datos antes del cálculo
if N1 <= k or N2 <= k:
    print(f"ERROR: No hay suficientes observaciones en uno de los subperíodos (N1={N1}, N2={N2}) para estimar {k} coeficientes.")
else:
    model_2 = sm.OLS(y2, X2).fit()
    RSS_2 = model_2.ssr
    
    # --- 4. CÁLCULO DEL ESTADÍSTICO DE CHOW ---
    
    numerator = (RSS_R - (RSS_1 + RSS_2)) / k
    denominator = (RSS_1 + RSS_2) / (N_total - 2 * k)
    
    F_statistic = numerator / denominator

    # Valor p: usando la distribución F
    p_value = f.sf(F_statistic, k, N_total - 2 * k)

    # --- 5. RESULTADOS ---
    alpha = 0.05

    print("--- Resultados del Test de Chow (Estabilidad Temporal) ---")
    print(f"Punto de Quiebre: {split_date.strftime('%Y-%m-%d')}")
    print(f"Número de Coeficientes (k): {k}")
    print(f"Estadístico F: {F_statistic:.4f}")
    print(f"Valor P: {p_value:.4f}")
    

Inestabilidad Confirmada: El Valor P de 0.0003 es significativamente menor que el nivel de significancia de 0.05. Por lo tanto, rechazamos la hipótesis nula (H 
0
​	
 ).

Cambio Estructural: Hay evidencia estadística fuerte de inestabilidad estructural en el modelo. Esto implica que la relación entre las variables de control (Peso, Servicio, Rutas) y el Costo, así como el Premium/Descuento de los Transportistas, cambiaron significativamente después del 31 de octubre.

Discusión Requerida: Este hallazgo obliga a una discusión profunda en la interpretación final:

Causa Potencial: El cambio estructural puede estar impulsado por eventos estacionales (preparación para la temporada alta de fin de año en Noviembre/Diciembre), cambios en la política de precios de la empresa, o la renegociación de tarifas con los transportistas clave.

Modelo Final: Sugiere que un único modelo para todo el período puede ser inadecuado. El análisis debería enfocarse en el período más reciente (Nov-Dic) o considerar la modelación con coeficientes variables en el tiempo.

### 4.Bootstrap

Estimar los intervalos de confianza del 95% para los coeficientes clave, como el Peso y el Transportista clave (Carrier_100139.0), mediante remuestreo para verificar si los resultados se mantienen significativos bajo un enfoque no paramétrico.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


# Creamos la matriz X completa 
X_full_data = pd.concat([
    df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']],
    route_dummies,
    carrier_dummies
], axis=1).astype(float)
X = sm.add_constant(X_full_data, has_constant='add')

N = len(X)
n_iterations = 500

# Identificar Carrier Clave y su índice (asumiendo Carrier_100139.0)
carrier_name = 'Carrier_100139.0'
carrier_col_index = X_full_data.columns.get_loc(carrier_name)
carrier_idx = 1 + carrier_col_index 

# Índice de la variable Peso
peso_idx = 1

# --- 2. MODELO BASE (Para obtener CIs estándar y coeficiente) ---
model_base = sm.OLS(y, X).fit()

# --- 3. EJECUCIÓN DEL BOOTSTRAP DE PARES (CÓDIGO CORREGIDO) ---
boot_coeffs = []

for i in range(n_iterations):
    # Remuestrear índices con reemplazo (Pairs Bootstrap)
    sample_indices = np.random.choice(N, size=N, replace=True)
    
    X_sample = X.iloc[sample_indices]
    y_sample = y[sample_indices] # <--- CORRECCIÓN APLICADA AQUÍ

    
    try:
        model_boot = sm.OLS(y_sample, X_sample).fit()
        
        # Almacenar los coeficientes de interés
        boot_coeffs.append({
            'Peso_Coef': model_boot.params[peso_idx],
            'Carrier_Coef': model_boot.params[carrier_idx]
        })
    except:
        continue

boot_df = pd.DataFrame(boot_coeffs)

# --- 4. CÁLCULO DE INTERVALOS DE CONFIANZA DEL BOOTSTRAP (Percentil) ---

# CIs de Bootstrap (Percentil 2.5 y 97.5)
peso_ci_boot = boot_df['Peso_Coef'].quantile([0.025, 0.975]).tolist()
carrier_ci_boot = boot_df['Carrier_Coef'].quantile([0.025, 0.975]).tolist()

# Resultados del modelo base para comparación (CIs estándar)
# Convertir model_base.conf_int() a DataFrame si es necesario para usar .iloc
ci_ols_df = model_base.conf_int()
peso_ci_ols = ci_ols_df.iloc[peso_idx].tolist()
carrier_ci_ols = ci_ols_df.iloc[carrier_idx].tolist()
carrier_coef_ols = model_base.params[carrier_idx]

# Crear tabla de resultados
results_table = pd.DataFrame({
    'Coeficiente': ['Peso Total (kg)', carrier_name],
    'Estimación OLS': [model_base.params[peso_idx], carrier_coef_ols],
    'CI OLS (95%)': [f"[{peso_ci_ols[0]:.2f}, {peso_ci_ols[1]:.2f}]", 
                     f"[{carrier_ci_ols[0]:.2f}, {carrier_ci_ols[1]:.2f}]"],
    'CI Bootstrap (95%)': [f"[{peso_ci_boot[0]:.2f}, {peso_ci_boot[1]:.2f}]", 
                           f"[{carrier_ci_boot[0]:.2f}, {carrier_ci_boot[1]:.2f}]"]
})

print("\n--- Resultados de Bootstrap (Intervalos de Confianza) ---")
print(f"Número de iteraciones válidas: {len(boot_df)}")
print(results_table.to_string(index=False))

Robustez del Transportista (Hallazgo Clave):

El Intervalo de Confianza del Bootstrap para el transportista clave es [−2,232.51,−1,294.25]. Este intervalo no contiene el cero, lo que confirma que el descuento promedio de $−1,794.87 es estadísticamente significativo al 95% bajo un enfoque de inferencia no paramétrica.

El CI Bootstrap es más estrecho que el CI OLS tradicional, indicando que el Premium es altamente preciso y robusto, incluso al considerar la variabilidad de la muestra.

Incertidumbre del Peso:

El coeficiente del Peso Total (kg) (0.03516) se encuentra en un área de marginalidad. Mientras que el CI OLS apenas excluye el cero, el CI Bootstrap [−0.00,0.07] apenas lo incluye, indicando que la contribución del peso al costo no es estadísticamente significativa con un nivel de confianza estricto.

### 5. Diagnósticos Específicos del Método

#### 5.1 Multicolinealidad (Factor de Inflación de la Varianza - VIF)

La multicolinealidad existe cuando las variables predictoras están altamente correlacionadas entre sí. Un VIF alto (VIF>5 o 10) infla los errores estándar, lo que dificulta determinar la contribución individual de cada variable.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

def calculate_vif(df):
    # Convertir a float explícitamente y manejar NaNs (CORRECCIÓN CLAVE)
    df = df.apply(pd.to_numeric, errors='coerce').replace([np.inf, -np.inf], np.nan).dropna()
    
    if df.shape[0] == 0:
        return pd.DataFrame({'Variable': ['N/A'], 'VIF': [np.nan]})

    vif_data = pd.DataFrame()
    vif_data["Variable"] = df.columns
    
    # Añadir constante temporalmente para el cálculo de VIF (necesario para la fórmula)
    X_temp = sm.add_constant(df, prepend=True)
    
    # Calcular VIF para cada variable
    vif_list = []
    try:
        for i in range(X_temp.shape[1]):
            vif_val = variance_inflation_factor(X_temp.values, i)
            vif_list.append(vif_val)
        
        vif_data["VIF"] = vif_list[1:] # Excluir el VIF de la constante (índice 0)
        vif_data["Variable"] = df.columns
        
    except Exception as e:
        print(f"Advertencia: Error al calcular VIF. {e}")
        vif_data["VIF"] = [np.nan] * len(df.columns)
        
    return vif_data.sort_values(by="VIF", ascending=False)

# Variables principales (Peso y Servicio)
main_vars = df_clean[['Peso Total (kg)', 'Es_Express', 'Es_Revestidos']].astype(float)
vif_main = calculate_vif(main_vars)

carrier_sample_cols = list(carrier_dummies.columns[:2])
route_sample_cols = list(route_dummies.columns[:3])

X_dummy_sample = pd.concat([main_vars] + 
                           [carrier_dummies[col] for col in carrier_sample_cols if col in carrier_dummies.columns] + 
                           [route_dummies[col] for col in route_sample_cols if col in route_dummies.columns], 
                           axis=1).astype(float)
vif_dummies = calculate_vif(X_dummy_sample)


print("\n--- Resultados de VIF (Variables Principales) ---")
print(vif_main.to_string(index=False, float_format='%.2f'))
print("\n--- Resultados de VIF (Muestra de Dummies) ---")
print(vif_dummies.to_string(index=False, float_format='%.2f'))

Variables Principales: Los valores VIF para todas las variables continuas, binarias y la muestra de dummies son extremadamente bajos (VIF<2).

Conclusión: Esto indica que no existe un problema significativo de multicolinealidad. La contribución individual de cada variable se puede medir de forma fiable, garantizando que los coeficientes del Peso y del Transportista no están sesgados por la alta correlación con otros predictores.

#### 5.2 Heterocedasticidad (Test de Breusch-Pagan)

Verificar si la varianza de los residuos del modelo es constante (homocedasticidad). La heterocedasticidad (varianza no constante) es muy común en análisis de costos, y si está presente, hace que los errores estándar de OLS sean incorrectos.

In [ ]:
from statsmodels.stats.api import het_breuschpagan
import statsmodels.api as sm

# Obtener los residuos y las variables explicativas
resid = model_base.resid
exog = model_base.model.exog # Usar las variables explicativas que se usaron para ajustar el modelo

# Ejecutar el test de Breusch-Pagan
# El test de Breusch-Pagan prueba la hipótesis nula de homocedasticidad.
bp_test_results = het_breuschpagan(resid, exog)

print("\n--- Resultados del Test de Breusch-Pagan (Heterocedasticidad) ---")
print(f"Estadístico LM: {bp_test_results[0]:.4f}")
print(f"P-valor LM: {bp_test_results[1]:.4f}")
print(f"Estadístico F: {bp_test_results[2]:.4f}")
print(f"P-valor F: {bp_test_results[3]:.4f}")

Hipótesis Nula (H 
0
​	
 ): Homocedasticidad (varianza constante).

Conclusión: Dado que el P-valor (tanto LM como F) es 0.0000 (menor a 0.05), rechazamos firmemente la hipótesis nula.

Implicación: Se confirma la presencia de heterocedasticidad. Esto significa que los errores del modelo son mayores para ciertos niveles de costo o variables explicativas. Los errores estándar tradicionales de OLS son incorrectos y, por lo tanto, los P-valores de tus coeficientes (incluyendo el del transportista) son inválidos para la inferencia.


#### 5.3 Autocorrelación (Test de Durbin-Watson)

Verificar si los errores del modelo están correlacionados en el tiempo (autocorrelación), lo cual es crucial en datos de series de tiempo.

In [ ]:
from statsmodels.stats.stattools import durbin_watson

# Se usa el modelo base (OLS) ajustado
# El test de Durbin-Watson opera sobre los residuos del modelo
dw_statistic = durbin_watson(model_base.resid)

print("\n--- Resultados del Test de Durbin-Watson (Autocorrelación) ---")
print(f"Estadístico Durbin-Watson: {dw_statistic:.4f}")

Interpretación: El Estadístico DW se encuentra muy cerca de 2.0, el valor ideal que indica la ausencia de autocorrelación de primer orden en los residuos.

Conclusión: Se concluye que la autocorrelación serial no es una preocupación principal para la inferencia estadística del modelo.

La confirmación de la Heterocedasticidad y la Inestabilidad Temporal exigen una corrección para garantizar que los P-valores de los coeficientes sean válidos y fiables.

El modelo final para la inferencia debe ser estimado utilizando Errores Estándar Robustos a la Heterocedasticidad (Huber-White). Dada la estructura de datos de panel y series de tiempo, la opción más segura es utilizar Errores Estándar HAC (Heteroskedasticity and Autocorrelation Consistent) (Newey-West), ya que corrigen la heterocedasticidad y cualquier potencial autocorrelación que el Test DW pudo haber fallado en detectar completamente.

In [ ]:
import pandas as pd
import numpy as np

params_series = pd.Series(model_full.params, index=['const'] + list(X_full_data.columns))
carrier_params = params_series[params_series.index.str.contains('Carrier_')]

best_carrier_coef = carrier_params.min()
best_carrier_id = carrier_params.idxmin() 
try:
    carrier_id_str = best_carrier_id.replace('Carrier_', '')
    carrier_id_float = float(carrier_id_str) 
    carrier_id_int = int(carrier_id_float) 

    if 'carrier_names_map' in locals():
        best_carrier_name_real = carrier_names_map.get(carrier_id_int, f"ID {carrier_id_int}")
    elif 'carrier_stats' in locals():
         best_carrier_name_real = carrier_stats[carrier_stats['Carrier_ID'] == carrier_id_int]['Carrier_Name'].iloc[0]
    else:
        best_carrier_name_real = best_carrier_id
except Exception as e:
    best_carrier_name_real = best_carrier_id 

print(f"🏆 Mejor Transportista: {best_carrier_name_real} ({best_carrier_id})")
print(f"   Descuento aplicado por viaje: ${best_carrier_coef:,.2f}")

current_carrier_costs = (carrier_dummies * carrier_params[carrier_dummies.columns].values).sum(axis=1)

counterfactual_carrier_costs = best_carrier_coef 

savings_per_trip = current_carrier_costs - counterfactual_carrier_costs
total_savings = savings_per_trip.sum()

actual_total_cost = np.sum(y) 
new_total_cost = actual_total_cost - total_savings
percent_saving = (total_savings / actual_total_cost) * 100

print("\n--- RESULTADOS ESCENARIO 1 ---")
print(f"Costo Total Actual:       ${actual_total_cost:,.2f}")
print(f"Costo Simulado (Best):    ${new_total_cost:,.2f}")
print(f"AHORRO TOTAL POTENCIAL:   ${total_savings:,.2f}")
print(f"Porcentaje de Ahorro:     {percent_saving:.2f}%")

In [ ]:
import pandas as pd
import numpy as np

if not isinstance(model_full.params, pd.Series):
    params_series = pd.Series(model_full.params, index=['const'] + list(X_full_data.columns))
else:
    params_series = model_full.params

route_params = params_series[params_series.index.str.contains('Route_')]

inefficient_routes = route_params.nlargest(5)

print(" Top 5 Rutas Más Ineficientes (Sobrecosto por viaje):")
print(inefficient_routes)

target_route_dummies = route_dummies[inefficient_routes.index]
savings_per_trip_s2 = target_route_dummies.dot(inefficient_routes)

total_savings_s2 = savings_per_trip_s2.sum()
actual_total_cost = np.sum(y)
new_total_cost_s2 = actual_total_cost - total_savings_s2
percent_saving_s2 = (total_savings_s2 / actual_total_cost) * 100

s2_low = total_savings_s2 * 0.95
s2_high = total_savings_s2 * 1.05

print("\n--- RESULTADOS ESCENARIO 2 ---")
print(f"Rutas Optimizadas:        {len(inefficient_routes)}")
print(f"Viajes Afectados:         {int((savings_per_trip_s2 > 0).sum())}")
print(f"AHORRO TOTAL POTENCIAL:   ${total_savings_s2:,.2f}")
print(f"Porcentaje de Ahorro:     {percent_saving_s2:.2f}%")

In [ ]:
import pandas as pd
import numpy as np
median_cost = carrier_params.median()

expensive_carriers = carrier_params[carrier_params > median_cost].index
efficient_carriers = carrier_params[carrier_params <= median_cost].index

print(f"Umbres de Costo (Mediana): ${median_cost:,.2f}")
print(f"Transportistas 'Caros':      {len(expensive_carriers)}")
print(f"Transportistas 'Eficientes': {len(efficient_carriers)}")

# Volumen de viajes por transportista
carrier_volumes = carrier_dummies.sum()

# Promedio ponderado del grupo "Caro"
vol_expensive = carrier_volumes[expensive_carriers].sum()
avg_cost_expensive = (carrier_params[expensive_carriers] * carrier_volumes[expensive_carriers]).sum() / vol_expensive

# Promedio ponderado del grupo "Eficiente"
vol_efficient = carrier_volumes[efficient_carriers].sum()
avg_cost_efficient = (carrier_params[efficient_carriers] * carrier_volumes[efficient_carriers]).sum() / vol_efficient

print(f"Costo Promedio 'Caros':      ${avg_cost_expensive:,.2f}")
print(f"Costo Promedio 'Eficientes': ${avg_cost_efficient:,.2f}")
print(f"Diferencia (Ahorro por viaje reasignado): ${avg_cost_expensive - avg_cost_efficient:,.2f}")

# 3. Simulación: Mover X% del volumen "Caro" al grupo "Eficiente"
shift_percentage = 0.30  # Mover el 30% de la carga
trips_to_shift = int(vol_expensive * shift_percentage)

# Ahorro Total = Viajes Reasignados * Ahorro por Viaje
total_savings_s3 = trips_to_shift * (avg_cost_expensive - avg_cost_efficient)

# Totales
actual_total_cost = np.sum(y)
new_total_cost_s3 = actual_total_cost - total_savings_s3
percent_saving_s3 = (total_savings_s3 / actual_total_cost) * 100

# Intervalo de incertidumbre (+/- 10% porque depende de la disponibilidad real)
s3_low = total_savings_s3 * 0.90
s3_high = total_savings_s3 * 1.10

print("\n--- RESULTADOS ESCENARIO 3 ---")
print(f"Política:                 Reasignar {shift_percentage*100}% del volumen caro")
print(f"Viajes Reasignados:       {trips_to_shift}")
print(f"AHORRO TOTAL POTENCIAL:   ${total_savings_s3:,.2f}")
print(f"Porcentaje de Ahorro:     {percent_saving_s3:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Configuración general de estilo
sns.set_theme(style="whitegrid")
plt.rcParams.update({'font.size': 12})

# ==========================================
# GRÁFICO 1: EL TECHO TÉCNICO (Escenario 1)
# Comparación de Costo Total: Actual vs Ideal
# ==========================================
labels_s1 = ['Costo Actual', 'Costo Simulado\n(Best-in-Class)']
values_s1 = [230.3, 203.4] # Millones MXN
ahorro_s1 = 26.9

plt.figure(figsize=(8, 6))
bars1 = plt.bar(labels_s1, values_s1, color=['#7f7f7f', '#2ca02c'], width=0.6)
plt.title('Escenario 1: Potencial Máximo de Ahorro', fontsize=14, fontweight='bold', pad=15)
plt.ylabel('Costo Total (Millones MXN)')
plt.ylim(0, 260)

# Etiquetas
for bar in bars1:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 2,
             f'${height:.1f}M', ha='center', va='bottom', fontweight='bold')

# Flecha de ahorro
plt.annotate(f'Ahorro:\n-${ahorro_s1}M (-11.7%)', 
             xy=(1, 203.4), xytext=(0.5, 220),
             arrowprops=dict(facecolor='black', arrowstyle='->', connectionstyle="arc3,rad=.2"),
             fontsize=11, ha='center', color='darkgreen', fontweight='bold')

plt.tight_layout()
plt.savefig('scenario1_benchmark.png', dpi=300)
plt.close()

# ==========================================
# GRÁFICO 2: RUTAS CRÍTICAS (Escenario 2)
# Sobrecosto Estructural por Ruta (Top 5)
# ==========================================
# Usamos los datos reales de tus rutas ineficientes
routes = ['NL - Juarez', 'NL - Puebla', 'NL - Tepeapulco', 'COA - Juarez', 'NL - CDMX']
# Estos son los coeficientes (sobrecosto por viaje) que me pasaste
overcosts = [47099, 42599, 38053, 35226, 34819] 

plt.figure(figsize=(10, 6))
bars2 = plt.barh(routes, overcosts, color='#d62728')
plt.title('Escenario 2: Sobrecosto Estructural por Viaje (Top 5 Rutas)', fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Sobrecosto Unitario vs Promedio (MXN)')
plt.gca().invert_yaxis() # Para que la #1 quede arriba

# Etiquetas dentro de las barras
for bar in bars2:
    width = bar.get_width()
    plt.text(width - 2000, bar.get_y() + bar.get_height()/2, 
             f'+${width:,.0f}', 
             va='center', ha='right', color='white', fontweight='bold')

plt.tight_layout()
plt.savefig('scenario2_routes.png', dpi=300)
plt.close()

# ==========================================
# GRÁFICO 3: ARBITRAJE DE COSTOS (Escenario 3)
# Diferencia de Precio Promedio: Grupo Caro vs Eficiente
# ==========================================
groups = ['Grupo Costoso', 'Grupo Eficiente']
avg_costs = [3604, -634] # Datos reales que calculaste
gap = 3604 - (-634)

plt.figure(figsize=(8, 6))
bars3 = plt.bar(groups, avg_costs, color=['#ff7f0e', '#1f77b4'], width=0.6)
plt.axhline(0, color='black', linewidth=0.8)
plt.title('Escenario 3: Disparidad de Costos (Oportunidad de Asignación)', fontsize=14, fontweight='bold', pad=15)
plt.ylabel('Costo Promedio Residual por Viaje (MXN)')

# Etiquetas
plt.text(0, 3604 + 200, '+$3,604', ha='center', fontweight='bold', color='#ff7f0e')
plt.text(1, -634 - 400, '-$634', ha='center', fontweight='bold', color='#1f77b4')

# Línea de brecha
plt.plot([0, 1], [3604, 3604], color='gray', linestyle='--')
plt.plot([1, 1], [3604, -634], color='gray', linestyle='--')
plt.text(1.1, 1500, f'Gap de Ahorro:\n${gap:,.0f} / viaje', va='center', color='gray', fontweight='bold')

plt.tight_layout()
plt.savefig('scenario3_allocation.png', dpi=300)
plt.close()

print("¡Listo! Se generaron 3 imágenes: scenario1_benchmark.png, scenario2_routes.png, scenario3_allocation.png")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# ======================================================================
# ANÁLISIS DE PARETO: RUTAS MÁS SOLICITADAS
# ======================================================================

# 1. Calcular frecuencia de viajes por ruta
# Usamos 'Ruta_ID' que creaste en el feature engineering
route_counts = df_clean['Ruta_ID'].value_counts().reset_index()
route_counts.columns = ['Ruta', 'Frecuencia']

# Calcular porcentaje acumulado (Pareto)
route_counts['Porcentaje'] = (route_counts['Frecuencia'] / route_counts['Frecuencia'].sum()) * 100
route_counts['Acumulado'] = route_counts['Porcentaje'].cumsum()

# Seleccionar el Top 15 para el gráfico
top_routes = route_counts.head(15)

# --- 2. VISUALIZACIÓN ---
plt.figure(figsize=(12, 8))
sns.barplot(x='Frecuencia', y='Ruta', data=top_routes, palette='viridis')

plt.title('Top 15 Rutas Más Frecuentes (Demanda de Viajes)', fontsize=14, fontweight='bold')
plt.xlabel('Número de Viajes', fontsize=12)
plt.ylabel('Ruta (Origen - Destino)', fontsize=12)
plt.grid(axis='x', linestyle=':', alpha=0.6)

# Añadir etiquetas de valor al final de las barras
for index, value in enumerate(top_routes['Frecuencia']):
    plt.text(value + 5, index, str(value), va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('top_rutas_frecuentes.png', dpi=300)
plt.show()

# --- 3. DATOS PARA EL REPORTE ---
top_5_share = route_counts.head(5)['Porcentaje'].sum()
top_10_share = route_counts.head(10)['Porcentaje'].sum()

print(f"\n--- ESTADÍSTICAS DE CONCENTRACIÓN ---")
print(f"Total de Rutas Únicas: {len(route_counts)}")
print(f"El Top 5 de rutas concentra el {top_5_share:.1f}% de todos los viajes.")
print(f"El Top 10 de rutas concentra el {top_10_share:.1f}% de todos los viajes.")
print("\nListado del Top 5:")
print(top_routes.head(5)[['Ruta', 'Frecuencia', 'Porcentaje']])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Datos del Business Case (Estimados en la sección anterior)
modes = ['Actual (Camión FTL)', 'Propuesta (Intermodal)']

# Desglose de costos
base_cost = [20000, 40300]      # Flete Base: El tren es más caro en tarifa base
inefficiency = [47099, 0]       # El camión tiene el sobrecosto de ruta, el tren no.
last_mile = [0, 8000]           # El tren requiere camión local (Drayage)

# Colores corporativos/serios
colors = ['#bfbfbf', '#d62728', '#2ca02c'] # Gris, Rojo (Ineficiencia), Verde (Last Mile)

fig, ax = plt.subplots(figsize=(8, 6))

# Crear barras apiladas
p1 = ax.bar(modes, base_cost, label='Tarifa Base', color='#1f77b4', width=0.5)
p2 = ax.bar(modes, inefficiency, bottom=base_cost, label='Sobrecosto Ruta (Ineficiencia)', color='#d62728', width=0.5)
p3 = ax.bar(modes, last_mile, bottom=np.array(base_cost)+np.array(inefficiency), label='Última Milla (Drayage)', color='#2ca02c', width=0.5)

# Etiquetas de totales
totals = [67100, 48300]
savings = 67100 - 48300
percent = (savings / 67100) * 100

# Poner los totales arriba de las barras
for i, total in enumerate(totals):
    ax.text(i, total + 1500, f'${total:,.0f}', ha='center', fontweight='bold', fontsize=12)

# Flecha de ahorro
ax.annotate(f'Ahorro por Viaje:\n${savings:,.0f} (-{percent:.0f}%)', 
            xy=(1, 48300), xytext=(0.5, 60000),
            arrowprops=dict(facecolor='black', arrowstyle='->', connectionstyle="arc3,rad=.2"),
            fontsize=11, ha='center', color='darkgreen', fontweight='bold')

# Formato
ax.set_ylabel('Costo Total por Viaje (MXN)')
ax.set_title('Análisis de Cambio Modal: Ruta NL - Juárez', fontweight='bold', pad=15)
ax.legend(loc='upper right')
ax.set_ylim(0, 80000)
ax.grid(axis='y', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('modal_comparison.png', dpi=300)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ======================================================================
# ANÁLISIS DE FRECUENCIA: ¿CADA CUÁNTO SALE UN CAMIÓN?
# ======================================================================

# Rutas a analizar (Las críticas que identificamos)
target_routes = ['NL - Juarez', 'NL - Área Metro Puebla', 'COA - Juarez']

print("--- ANÁLISIS DE FRECUENCIA DE SALIDAS ---\n")

for ruta in target_routes:
    # 1. Filtrar datos de la ruta y ordenar por fecha
    # Asumimos que 'Ruta_ID' es la columna de origen-destino
    trips = df_clean[df_clean['Ruta_ID'] == ruta].sort_values('F.Salida').copy()
    
    if len(trips) > 1:
        # 2. Calcular tiempo entre viajes consecutivos (Gap)
        trips['Prev_Salida'] = trips['F.Salida'].shift(1)
        trips['Gap_Horas'] = (trips['F.Salida'] - trips['Prev_Salida']).dt.total_seconds() / 3600
        
        # Estadísticas
        avg_gap = trips['Gap_Horas'].mean()
        median_gap = trips['Gap_Horas'].median()
        trips_per_week = len(trips) / ( (trips['F.Salida'].max() - trips['F.Salida'].min()).days / 7 )
        
        print(f"RUTA: {ruta}")
        print(f"   Total Viajes: {len(trips)}")
        print(f"   Salidas por Semana (Promedio): {trips_per_week:.1f}")
        print(f"   Tiempo Promedio entre salidas: {avg_gap:.1f} horas ({avg_gap/24:.1f} días)")
        print(f"   Mediana de espera:             {median_gap:.1f} horas")
        print("-" * 40)
        
    else:
        print(f"RUTA: {ruta} - Datos insuficientes ({len(trips)} viajes)")

# --- VISUALIZACIÓN: DÍAS DE SALIDA ---
# Ver si hay un patrón semanal (ej. ¿Solo salen los viernes?)
subset = df_clean[df_clean['Ruta_ID'].isin(target_routes)].copy()
subset['Dia_Semana'] = subset['F.Salida'].dt.day_name()

plt.figure(figsize=(10, 6))
sns.countplot(data=subset, x='Dia_Semana', hue='Ruta_ID', 
              order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Patrón de Salidas por Día de la Semana', fontweight='bold')
plt.ylabel('Número de Viajes')
plt.grid(axis='y', linestyle=':', alpha=0.5)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ======================================================================
# ANÁLISIS DE FRECUENCIA REAL: SOLO VIAJES STANDARD (NO EXPRESS)
# ======================================================================

# 1. FILTRO CRÍTICO: Solo analizamos lo que se puede "aguantar" (Standard)
# Asumimos que 'Es_Express' == 0 significa Standard
standard_trips = df_clean[df_clean['Es_Express'] == 0].copy()

print(f"Total Viajes Standard en la red: {len(standard_trips)}")
print("--- FRECUENCIA DE SALIDAS (EXCLUYENDO EXPRESS) ---\n")

target_routes = ['NL - Juarez', 'NL - Área Metro Puebla', 'COA - Juarez']

for ruta in target_routes:
    # Filtrar datos de la ruta y ordenar por fecha
    trips = standard_trips[standard_trips['Ruta_ID'] == ruta].sort_values('F.Salida').copy()
    
    if len(trips) > 1:
        # Calcular tiempo entre viajes consecutivos
        trips['Prev_Salida'] = trips['F.Salida'].shift(1)
        trips['Gap_Horas'] = (trips['F.Salida'] - trips['Prev_Salida']).dt.total_seconds() / 3600
        
        # Estadísticas
        avg_gap = trips['Gap_Horas'].mean()
        median_gap = trips['Gap_Horas'].median()
        # Días del periodo analizado (aprox)
        days_span = (trips['F.Salida'].max() - trips['F.Salida'].min()).days
        trips_per_week = len(trips) / (days_span / 7) if days_span > 0 else 0
        
        print(f"RUTA: {ruta}")
        print(f"   Total Viajes Standard: {len(trips)}")
        print(f"   Salidas por Semana:    {trips_per_week:.1f}")
        print(f"   Tiempo Promedio Gap:   {avg_gap:.1f} horas ({avg_gap/24:.1f} días)")
        print(f"   Mediana de espera:     {median_gap:.1f} horas")
        
        # Interpretación automática
        if median_gap <= 24:
            print("   ✅ VIABLE: Sale al menos un camión diario.")
        elif median_gap <= 48:
            print("   ⚠️ RIESGO MEDIO: Sale cada 2 días. Esperar 24h podría retrasar 48h.")
        else:
            print("   ❌ NO VIABLE: Frecuencia muy baja para estrategia de espera.")
        print("-" * 40)
        
    else:
        print(f"RUTA: {ruta} - Datos insuficientes ({len(trips)} viajes)")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# ======================================================================
# ANÁLISIS DE CONSOLIDACIÓN: ¿VIAJAN LLENOS O VACÍOS?
# ======================================================================

# 1. Definir Capacidad Teórica (Asumimos Caja de 53' = 24 Toneladas aprox)
# Si tus datos tienen otro máximo, el código se ajustará al máximo observado si es mayor.
CAPACIDAD_KG = 24000 

# Rutas Críticas a analizar
target_routes = ['NL - Juarez', 'NL - Área Metro Puebla', 'COA - Juarez']
# Sus sobrecostos fijos (del modelo econométrico)
route_overcosts = {
    'NL - Juarez': 47099, 
    'NL - Área Metro Puebla': 42599,
    'COA - Juarez': 35225
}

print(f"--- ANÁLISIS DE EFICIENCIA DE CARGA (Capacidad ref: {CAPACIDAD_KG/1000} Tons) ---")

consolidation_data = []

for ruta in target_routes:
    # Filtrar viajes de esa ruta
    trips = df_clean[df_clean['Ruta_ID'] == ruta].copy()
    
    if len(trips) > 0:
        # Calcular % de Ocupación
        # Si hay pesos mayores a 24k, usamos el real, si no, el tope de 24k
        trips['Ocupacion'] = trips['Peso Total (kg)'] / CAPACIDAD_KG * 100
        # Capar al 100% para visualización (si hubo sobrepeso)
        trips['Ocupacion_Vis'] = trips['Ocupacion'].clip(upper=100)
        
        # Calcular "Costo de Ineficiencia por Kg"
        # Cuánto pagamos de sobrecosto por cada kilo real movido
        overcost = route_overcosts.get(ruta, 0)
        trips['Ineficiencia_x_Kg'] = overcost / trips['Peso Total (kg)']
        
        # Oportunidad de Consolidación (Viajes < 60% ocupación)
        low_utilization = trips[trips['Ocupacion'] < 60]
        potential_trips = len(low_utilization)
        
        print(f"\nRUTA: {ruta}")
        print(f"   Viajes Totales: {len(trips)}")
        print(f"   Peso Promedio:  {trips['Peso Total (kg)'].mean():,.0f} kg")
        print(f"   Ocupación Prom.: {trips['Ocupacion'].mean():.1f}%")
        print(f"   Viajes < 60% carga: {potential_trips} ({potential_trips/len(trips)*100:.1f}%) -> CANDIDATOS A JUNTAR")
        print(f"   Ineficiencia Promedio por Kg transportado: ${trips['Ineficiencia_x_Kg'].mean():.2f}/kg")

        consolidation_data.append(trips)

# --- VISUALIZACIÓN 1: HISTOGRAMA DE OCUPACIÓN ---
# ¿Qué tan llenos van los camiones?
all_trips_viz = pd.concat(consolidation_data)

plt.figure(figsize=(10, 6))
sns.histplot(data=all_trips_viz, x='Peso Total (kg)', hue='Ruta_ID', element="step", bins=20, alpha=0.6)
plt.axvline(CAPACIDAD_KG, color='red', linestyle='--', label='Capacidad Máxima (24 Ton)')
plt.axvline(CAPACIDAD_KG * 0.5, color='orange', linestyle='--', label='50% Capacidad (Punto de Fusión)')

plt.title('Distribución de Peso por Viaje en Rutas Críticas', fontweight='bold')
plt.xlabel('Peso Cargado (Kg)')
plt.ylabel('Frecuencia de Viajes')
plt.legend()
plt.grid(axis='y', linestyle=':', alpha=0.5)
plt.show()

# --- CÁLCULO DE AHORRO POR CONSOLIDACIÓN ---
# Si juntamos dos viajes de <12 toneladas en uno solo, nos ahorramos UN SOBRECOSTO ENTERO ($47k)
total_savings_consolidation = 0
trips_eliminated = 0

for ruta in target_routes:
    trips = df_clean[df_clean['Ruta_ID'] == ruta]
    # Viajes que son "medio camión" o menos
    light_trips = trips[trips['Peso Total (kg)'] <= (CAPACIDAD_KG * 0.55)] # 55% como margen
    
    # Cada par de viajes ligeros = 1 viaje eliminado
    pairs = int(len(light_trips) / 2)
    
    if pairs > 0:
        route_saving = pairs * route_overcosts[ruta] # Ahorramos el flete completo del camión eliminado
        total_savings_consolidation += route_saving
        trips_eliminated += pairs
        print(f"   > {ruta}: Podríamos fusionar {len(light_trips)} viajes ligeros en {len(light_trips)-pairs} viajes.")
        print(f"     Ahorro: ${route_saving:,.0f} ({pairs} viajes eliminados)")

print(f"\n💰 AHORRO TOTAL POTENCIAL POR CONSOLIDACIÓN: ${total_savings_consolidation:,.2f}")

In [ ]:
import pandas as pd
import numpy as np

# ======================================================================
# ANÁLISIS DE CAPACIDAD OCIOSA ("FLETE FALSO EN KG")
# ======================================================================

# 1. FILTRO: Solo viajes Standard (Los Express no se pueden consolidar por tiempo)
df_ops = df_clean[df_clean['Es_Express'] == 0].copy()

# 2. DEFINIR CAPACIDAD DE REFERENCIA
# Asumimos una caja estándar de 53 pies = 24,000 kg de capacidad útil
CAPACIDAD_REF = 24000 

# 3. CALCULAR "KG DE FLETE FALSO" (AIRE) POR VIAJE
# Si el camión lleva 10,000 kg, lleva 14,000 kg de "Flete Falso" (espacio pagado no usado)
# Si lleva más de 24,000 (sobrepeso/full), el flete falso es 0.
df_ops['Kg_Flete_Falso'] = (CAPACIDAD_REF - df_ops['Peso Total (kg)']).clip(lower=0)

# Identificar viajes "Consolidables" (Carga < 50% de capacidad)
df_ops['Es_Consolidable'] = df_ops['Peso Total (kg)'] <= (CAPACIDAD_REF * 0.55) # 55% margen

# 4. AGRUPAR POR RUTA
route_analysis = df_ops.groupby('Ruta_ID').agg({
    'Viaje': 'count',
    'Peso Total (kg)': 'mean',
    'Kg_Flete_Falso': ['sum', 'mean'],
    'Es_Consolidable': 'sum' # Cuántos viajes iban a menos de la mitad
}).reset_index()

# Aplanar nombres de columnas
route_analysis.columns = ['Ruta', 'Total_Viajes', 'Peso_Promedio', 'Total_Kg_Aire', 'Promedio_Kg_Aire', 'Viajes_Consolidables']

# 5. FILTRAR Y ORDENAR
# Solo rutas con volumen relevante (> 5 viajes) para que valga la pena la logística
route_analysis = route_analysis[route_analysis['Total_Viajes'] > 5]

# ORDENAR POR: ¿Dónde hay más Kilos de Aire Totales? (Mayor oportunidad de ahorro masivo)
top_empty_routes = route_analysis.sort_values('Total_Kg_Aire', ascending=False).head(10)

# Calcular % de Ineficiencia de Carga
top_empty_routes['%_Desperdicio'] = (top_empty_routes['Total_Kg_Aire'] / (top_empty_routes['Total_Viajes'] * CAPACIDAD_REF)) * 100

print("\n--- TOP 10 RUTAS CON MÁS 'FLETE FALSO' EN KG (OPORTUNIDAD DE CONSOLIDACIÓN) ---")
print(top_empty_routes[['Ruta', 'Total_Viajes', 'Viajes_Consolidables', 'Peso_Promedio', 'Promedio_Kg_Aire', '%_Desperdicio']])

# --- CÁLCULO DE AHORRO POTENCIAL ---
# Cada 2 viajes consolidables = 1 Flete ahorrado
# Estimamos un costo promedio por flete de $25,000 (conservador)
ahorro_estimado = (top_empty_routes['Viajes_Consolidables'].sum() / 2) * 25000

print(f"\n💡 OPORTUNIDAD: En estas 10 rutas hay {int(top_empty_routes['Viajes_Consolidables'].sum())} viajes que iban a menos de la mitad.")
print(f"   Si consolidamos pares, podríamos eliminar aprox. {int(top_empty_routes['Viajes_Consolidables'].sum()/2)} viajes.")
print(f"   Ahorro estimado (Fletes evitados): ${ahorro_estimado:,.0f} MXN")

In [ ]:
import pandas as pd

# ======================================================================
# VALIDACIÓN DE FRECUENCIA PARA RUTAS CON "AIRE" (Solo Standard)
# ======================================================================

# 1. Definir las Rutas Candidatas (Top 10 del análisis anterior)
target_consolidation_routes = [
    'NL - Área Metro SLP',
    'NL - Área Metro Saltillo',
    'NL - Área Metro Monclova',
    'NL - Area Metr Queretaro',
    'NL - Juarez',
    'NL - Edo. y Cd. de México',
    'NL - Área Metro Puebla',
    'NL - Altamira',
    'NL - Área Metro Celaya',
    'NL - A. M. Aguascalientes'
]

# 2. Filtro: Solo Viajes Standard
df_standard = df_clean[df_clean['Es_Express'] == 0].copy()

print("--- FRECUENCIA DE SALIDAS: RUTAS CANDIDATAS A CONSOLIDACIÓN ---\n")

for ruta in target_consolidation_routes:
    # Filtrar y ordenar
    trips = df_standard[df_standard['Ruta_ID'] == ruta].sort_values('F.Salida').copy()
    
    if len(trips) > 1:
        # Calcular Gaps
        trips['Prev_Salida'] = trips['F.Salida'].shift(1)
        trips['Gap_Horas'] = (trips['F.Salida'] - trips['Prev_Salida']).dt.total_seconds() / 3600
        
        # Métricas
        median_gap = trips['Gap_Horas'].median()
        trips_per_day = len(trips) / ((trips['F.Salida'].max() - trips['F.Salida'].min()).days)
        
        print(f"RUTA: {ruta}")
        print(f"   Viajes Standard: {len(trips)}")
        print(f"   Salidas Diarias Promedio: {trips_per_day:.1f}")
        print(f"   Tiempo Mediano de Espera: {median_gap:.1f} horas")
        
        # Veredicto de Consolidación
        if median_gap <= 12:
            print("   ✅ OPORTUNIDAD ALTA: Salen varios al día. ¡Consolidar es fácil!")
        elif median_gap <= 24:
            print("   ✅ OPORTUNIDAD MEDIA: Sale uno diario. Se puede esperar al día siguiente.")
        else:
            print("   ⚠️ OPORTUNIDAD BAJA: Frecuencia baja. Esperar implica retrasos de >1 día.")
        print("-" * 40)

In [ ]:
import pandas as pd
import numpy as np

# ======================================================================
# ANÁLISIS DE "FLETE FALSO PROMEDIO" (La Verdadera Ineficiencia de Carga)
# ======================================================================

# 1. Configuración
CAPACIDAD_REF = 24000 # 24 Toneladas
df_ops = df_clean[df_clean['Es_Express'] == 0].copy() # Solo viajes normales

# 2. Calcular Kilos Vacíos por Viaje individual
# Si trae más de 24k, es 0 vacío. Si trae menos, es la diferencia.
df_ops['Kg_Vacios'] = (CAPACIDAD_REF - df_ops['Peso Total (kg)']).clip(lower=0)
df_ops['%_Llenado'] = (df_ops['Peso Total (kg)'] / CAPACIDAD_REF) * 100

# 3. Agrupar por Ruta (Promedios)
route_avg = df_ops.groupby('Ruta_ID').agg({
    'Viaje': 'count',
    'Peso Total (kg)': 'mean',
    'Kg_Vacios': 'mean',     # <--- ESTA ES LA CLAVE
    '%_Llenado': 'mean'
}).reset_index()

route_avg.columns = ['Ruta', 'Frecuencia', 'Peso_Promedio', 'Aire_Promedio_x_Viaje', 'Llenado_Promedio']

# 4. Filtro de Relevancia
# Quitamos rutas "fantasma" con menos de 5 viajes para no ensuciar el análisis
route_avg = route_avg[route_avg['Frecuencia'] > 5]

# 5. Ordenar por el CAMIÓN MÁS VACÍO PROMEDIO
top_empty_avg = route_avg.sort_values('Aire_Promedio_x_Viaje', ascending=False).head(10)

print("\n--- TOP 10 RUTAS CON MAYOR ESPACIO VACÍO PROMEDIO ---")
print("(Rutas donde sistemáticamente desaprovechamos capacidad)")
print(top_empty_avg[['Ruta', 'Frecuencia', 'Llenado_Promedio', 'Aire_Promedio_x_Viaje']])

# Validación rápida
worst_route = top_empty_avg.iloc[0]
print(f"\n💡 HALLAZGO: En la ruta '{worst_route['Ruta']}', cada camión sale en promedio con {worst_route['Aire_Promedio_x_Viaje']:,.0f} kg de aire.")
print(f"   ¡Eso es un {100 - worst_route['Llenado_Promedio']:.1f}% de capacidad desperdiciada por viaje!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ======================================================================
# ANÁLISIS DE "RIGHT-SIZING": BAJAR DE FULL A SENCILLO
# ======================================================================

# 1. CONFIGURACIÓN
CAPACIDAD_SENCILLO = 22000  # 22 Toneladas (límite seguro para Sencillo)
PESO_MIN_FULL = 35000       # Si pesa más de esto, SEGURO es Full
AHORRO_ESTIMADO_PCT = 0.20  # Un Sencillo suele cobrar 20% menos que un Full

# Filtramos solo viajes Standard (No Express)
df_eq = df_clean[df_clean['Es_Express'] == 0].copy()

print(f"--- ANÁLISIS DE OPORTUNIDAD: CAMBIO DE EQUIPO (FULL -> SENCILLO) ---\n")

downgrade_opportunities = []

# Analizamos ruta por ruta para comparar peras con peras
for ruta in df_eq['Ruta_ID'].unique():
    route_trips = df_eq[df_eq['Ruta_ID'] == ruta]
    
    # Solo analizamos rutas con suficiente volumen y mezcla de pesos
    if len(route_trips) > 10:
        # 2. IDENTIFICAR EL COSTO DE UN "FULL" EN ESTA RUTA
        # Tomamos el promedio de los viajes pesados (>35 tons)
        heavy_trips = route_trips[route_trips['Peso Total (kg)'] > PESO_MIN_FULL]
        
        if len(heavy_trips) > 0:
            avg_cost_full = heavy_trips['Costo'].mean()
            
            # 3. BUSCAR VIAJES LIGEROS PERO CAROS
            # Ligeros: Caben en un Sencillo (< 22 tons)
            # Caros: Costaron casi lo mismo que un Full (> 85% del costo Full)
            # Esto indica que probablemente SE PAGÓ como Full aunque iba ligero
            inefficient_trips = route_trips[
                (route_trips['Peso Total (kg)'] <= CAPACIDAD_SENCILLO) & 
                (route_trips['Costo'] > (avg_cost_full * 0.85))
            ]
            
            num_ineff = len(inefficient_trips)
            
            if num_ineff > 0:
                # Calcular ahorro: Si hubiéramos pedido Sencillo, costaría 20% menos
                potential_saving = inefficient_trips['Costo'].sum() * AHORRO_ESTIMADO_PCT
                
                downgrade_opportunities.append({
                    'Ruta': ruta,
                    'Viajes_Candidatos': num_ineff,
                    'Costo_Promedio_Actual': inefficient_trips['Costo'].mean(),
                    'Costo_Ref_Full': avg_cost_full,
                    'Ahorro_Total': potential_saving
                })

# 4. RESULTADOS
if len(downgrade_opportunities) > 0:
    df_downgrade = pd.DataFrame(downgrade_opportunities).sort_values('Ahorro_Total', ascending=False)
    
    print(df_downgrade[['Ruta', 'Viajes_Candidatos', 'Costo_Ref_Full', 'Ahorro_Total']].head(10))
    
    total_saving_eq = df_downgrade['Ahorro_Total'].sum()
    total_trips_eq = df_downgrade['Viajes_Candidatos'].sum()
    
    print(f"\n💡 HALLAZGO: Encontramos {total_trips_eq} viajes que se pagaron a precio de 'Full' pero llevaban carga de 'Sencillo'.")
    print(f"💰 AHORRO POTENCIAL (Bajando a Sencillo): ${total_saving_eq:,.2f}")
else:
    print("No se detectaron oportunidades claras de cambio de equipo (La asignación actual parece eficiente).")

# --- VISUALIZACIÓN: SCATTER PLOT COSTO vs PESO ---
# Esto te ayuda a ver visualmente los dos grupos
top_route = df_downgrade.iloc[0]['Ruta'] if len(downgrade_opportunities) > 0 else target_routes[0]
viz_data = df_eq[df_eq['Ruta_ID'] == top_route]

plt.figure(figsize=(10, 6))
sns.scatterplot(data=viz_data, x='Peso Total (kg)', y='Costo', alpha=0.6)
plt.axvline(CAPACIDAD_SENCILLO, color='green', linestyle='--', label='Límite Sencillo (22 Ton)')
plt.axhline(viz_data[viz_data['Peso Total (kg)'] > PESO_MIN_FULL]['Costo'].mean(), color='red', linestyle='--', label='Costo Promedio Full')
plt.title(f'Análisis de Equipo: Ruta {top_route}', fontweight='bold')
plt.xlabel('Peso (Kg)')
plt.ylabel('Costo ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
import pandas as pd
import statsmodels.api as sm

# ======================================================================
# CORROBORACIÓN ECONOMÉTRICA: RUTA NL - JUAREZ
# ======================================================================

# 1. Buscar el coeficiente específico en tu modelo
# El nombre interno de la variable es 'Route_NL - Juarez'
target_variable = 'Route_NL - Juarez'

try:
    # Extraer el valor del coeficiente (El costo extra)
    beta_ruta = model_full.params[model_full.params.index.str.contains(target_variable)].iloc[0]
    
    # Extraer el P-Value (La certeza de que no es suerte)
    p_value = model_full.pvalues[model_full.pvalues.index.str.contains(target_variable)].iloc[0]
    
    # Extraer el Intervalo de Confianza (El rango real del golpe)
    conf_int = model_full.conf_int().loc[model_full.params.index.str.contains(target_variable)].iloc[0]

    print(f"--- RADIOGRAFÍA ECONOMÉTRICA: {target_variable} ---\n")
    print(f"1. COEFICIENTE (Sobrecosto):  ${beta_ruta:,.2f}")
    print(f"   (Esto es lo que cuesta de más ir a Juárez vs. una ruta promedio,")
    print(f"    después de descontar peso y transportista).")
    
    print(f"\n2. SIGNIFICANCIA ESTADÍSTICA (P-Value): {p_value:.10f}")
    if p_value < 0.05:
        print("   ✅ RESULTADO: Altamente Significativo. El sobrecosto es REAL y estructural.")
    else:
        print("   ❌ RESULTADO: No significativo (Podría ser ruido estadístico).")
        
    print(f"\n3. INTERVALO DE CONFIANZA (95%):")
    print(f"   Mínimo: ${conf_int[0]:,.2f}")
    print(f"   Máximo: ${conf_int[1]:,.2f}")
    print("   (Incluso en el mejor de los casos, el sobrecosto es brutal).")

except IndexError:
    print(f"No se encontró la variable '{target_variable}' en el modelo. Verifica el nombre exacto.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ======================================================================
# 1. CONFIGURACIÓN Y CÁLCULO
# ======================================================================
CAPACIDAD_CAMION = 24000  # 24 Toneladas por camión
df_daily = df_clean[df_clean['Es_Express'] == 0].copy() # Solo viajes Standard
df_daily['Fecha'] = df_daily['F.Salida'].dt.date 

# Agrupar: Sumar todo el peso que salió el mismo día hacia el mismo destino
daily_ops = df_daily.groupby(['Ruta_ID', 'Fecha']).agg({
    'Peso Total (kg)': 'sum',
    'Costo': 'sum',
    'Viaje': 'count'
}).reset_index()

# Simulación: ¿Cuántos camiones hubiéramos necesitado si llenáramos al tope?
daily_ops['Camiones_Optimos'] = np.ceil(daily_ops['Peso Total (kg)'] / CAPACIDAD_CAMION)
daily_ops['Viajes_Ahorrados'] = daily_ops['Viaje'] - daily_ops['Camiones_Optimos']

# Calcular dinero ahorrado (Costo Promedio del Día * Viajes Ahorrados)
daily_ops['Costo_Promedio_Unitario'] = daily_ops['Costo'] / daily_ops['Viaje']
daily_ops['Ahorro_MXN'] = daily_ops['Viajes_Ahorrados'] * daily_ops['Costo_Promedio_Unitario']

# ======================================================================
# 2. RESULTADOS FINANCIEROS
# ======================================================================
total_saving = daily_ops['Ahorro_MXN'].sum()
total_spend = df_clean['Costo'].sum()
trips_saved = daily_ops['Viajes_Ahorrados'].sum()
pct_impact = (total_saving / total_spend) * 100

print(f"--- REPORTE DE ESTRATEGIA: CONSOLIDACIÓN DINÁMICA ---")
print(f"Objetivo: Eliminar viajes redundantes llenando camiones al 100% diario.")
print(f"\n💰 RESULTADOS PROYECTADOS:")
print(f"   Ahorro Total Anualizado:  ${total_saving:,.2f} MXN")
print(f"   Impacto en Gasto Total:   -{pct_impact:.1f}%")
print(f"   Viajes Eliminados:        {int(trips_saved)} fletes menos")

# ======================================================================
# 3. VISUALIZACIÓN: ACTUAL VS OPTIMIZADO (TOP 5 RUTAS)
# ======================================================================
# Agrupar por ruta para el gráfico
route_summary = daily_ops.groupby('Ruta_ID').agg({
    'Viaje': 'sum',
    'Camiones_Optimos': 'sum',
    'Ahorro_MXN': 'sum'
}).sort_values('Ahorro_MXN', ascending=False).head(5)

# Preparar datos para plot
rutas = [r.replace('Route_', '').replace('NL - ', '') for r in route_summary.index] # Nombres cortos
actuales = route_summary['Viaje'].values
optimos = route_summary['Camiones_Optimos'].values

x = np.arange(len(rutas))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width/2, actuales, width, label='Viajes Actuales (Fragmentados)', color='#7f7f7f')
rects2 = ax.bar(x + width/2, optimos, width, label='Viajes Necesarios (Consolidados)', color='#2ca02c')

ax.set_ylabel('Número de Viajes')
ax.set_title('Eficiencia de Flota: Reducción de Viajes por Ruta (Top 5)', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(rutas, rotation=15)
ax.legend()

# Etiquetas de valor
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{int(height)}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontweight='bold')

autolabel(rects1)
autolabel(rects2)

# Anotación de ahorro en la mejor ruta
best_route_saving = route_summary.iloc[0]['Ahorro_MXN']
ax.text(0, actuales[0]*0.5, f"Ahorro:\n${best_route_saving/1000000:.1f}M", 
        ha='center', color='white', fontweight='bold', fontsize=11,
        bbox=dict(facecolor='green', alpha=0.7, edgecolor='none'))

plt.tight_layout()
plt.savefig('consolidacion_impacto.png', dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# ======================================================================
# 1. PREPARAR DATOS (Recalculando para graficar)
# ======================================================================
CAPACIDAD_CAMION = 24000
df_daily = df_clean[df_clean['Es_Express'] == 0].copy()
df_daily['Fecha'] = df_daily['F.Salida'].dt.date 

daily_ops = df_daily.groupby(['Ruta_ID', 'Fecha']).agg({
    'Peso Total (kg)': 'sum',
    'Viaje': 'count'
}).reset_index()

daily_ops['Camiones_Optimos'] = np.ceil(daily_ops['Peso Total (kg)'] / CAPACIDAD_CAMION)
daily_ops['Viajes_Ahorrados'] = daily_ops['Viaje'] - daily_ops['Camiones_Optimos']

# Agrupar por ruta para el Top 5
route_viz = daily_ops.groupby('Ruta_ID').agg({
    'Viaje': 'sum',
    'Camiones_Optimos': 'sum',
    'Viajes_Ahorrados': 'sum'
}).sort_values('Viajes_Ahorrados', ascending=False).head(5)

# Limpiar nombres para la gráfica
rutas = [r.replace('NL - Área Metro ', '').replace('NL - ', '') for r in route_viz.index]
actuales = route_viz['Viaje'].values
optimos = route_viz['Camiones_Optimos'].values
ahorros = route_viz['Viajes_Ahorrados'].values

# ======================================================================
# 2. GENERAR GRÁFICA
# ======================================================================
x = np.arange(len(rutas))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 7))

# Barras
rects1 = ax.bar(x - width/2, actuales, width, label='Viajes Actuales (Fragmentados)', color='#d62728', alpha=0.8)
rects2 = ax.bar(x + width/2, optimos, width, label='Viajes Necesarios (Consolidados)', color='#2ca02c', alpha=0.9)

# Formato
ax.set_ylabel('Número de Viajes Semestrales', fontsize=12)
ax.set_title('Impacto de Consolidación: Reducción de Frecuencia en Top 5 Rutas', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(rutas, fontsize=11)
ax.legend(fontsize=11)
ax.grid(axis='y', linestyle='--', alpha=0.3)

# Etiquetas de valor
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{int(height)}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontweight='bold')

autolabel(rects1)
autolabel(rects2)

# Flechas de Ahorro (El "Wow Factor")
for i in range(len(rutas)):
    # Calcular % de reducción
    pct = (ahorros[i] / actuales[i]) * 100
    # Poner texto entre las barras
    ax.text(i, optimos[i] + (actuales[i]-optimos[i])/2, 
            f'Ahorro:\n-{int(ahorros[i])} Viajes\n(-{pct:.0f}%)', 
            ha='center', va='center', color='black', fontsize=10, fontweight='bold',
            bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.3', alpha=0.8))

plt.tight_layout()
plt.savefig('impacto_consolidacion.png', dpi=300)
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# ======================================================================
# AUDITORÍA DE DATOS: VERIFICACIÓN DE OPORTUNIDADES REALES
# ======================================================================

print("--- 1. IDENTIFICANDO DÍAS CON MAYOR DESPERDICIO ---")

# 1. Preparar datos
CAPACIDAD_CAMION = 24000
df_audit = df_clean[df_clean['Es_Express'] == 0].copy()
df_audit['Fecha'] = df_audit['F.Salida'].dt.date

# 2. Calcular eficiencia por día
daily_audit = df_audit.groupby(['Ruta_ID', 'Fecha']).agg({
    'Viaje': 'count',
    'Peso Total (kg)': ['sum', 'mean'],
    'Costo': 'sum'
}).reset_index()

# Aplanar columnas
daily_audit.columns = ['Ruta', 'Fecha', 'Viajes_Reales', 'Peso_Total_Dia', 'Peso_Promedio_Viaje', 'Costo_Total_Dia']

# Calcular escenario ideal
daily_audit['Camiones_Necesarios'] = np.ceil(daily_audit['Peso_Total_Dia'] / CAPACIDAD_CAMION)
daily_audit['Desperdicio_Viajes'] = daily_audit['Viajes_Reales'] - daily_audit['Camiones_Necesarios']
daily_audit['Llenado_Promedio_Dia'] = (daily_audit['Peso_Total_Dia'] / (daily_audit['Viajes_Reales'] * CAPACIDAD_CAMION)) * 100

# Filtrar solo donde hubo desperdicio real ( > 0 viajes extra)
inefficient_days = daily_audit[daily_audit['Desperdicio_Viajes'] > 0].sort_values('Desperdicio_Viajes', ascending=False)

print(f"Se encontraron {len(inefficient_days)} días/ruta con viajes innecesarios.")
print("\nTop 5 Días con Mayor Ineficiencia (Ejemplos para mostrar):")
print(inefficient_days[['Ruta', 'Fecha', 'Viajes_Reales', 'Camiones_Necesarios', 'Desperdicio_Viajes', 'Llenado_Promedio_Dia']].head(5))

# --- 2. AUTOPSIA DE UN CASO REAL (DRILL-DOWN) ---
if len(inefficient_days) > 0:
    # Tomamos el peor caso para analizarlo
    worst_case = inefficient_days.iloc[0]
    target_route = worst_case['Ruta']
    target_date = worst_case['Fecha']
    
    print(f"\n\n--- 2. EVIDENCIA DETALLADA: ¿QUÉ PASÓ EL {target_date} EN {target_route}? ---")
    print(f"Resumen: Salieron {int(worst_case['Viajes_Reales'])} camiones. Con {worst_case['Peso_Total_Dia']:,.0f} kg totales, solo necesitábamos {int(worst_case['Camiones_Necesarios'])}.")
    print(f"¡Pagamos {int(worst_case['Desperdicio_Viajes'])} fletes de más ese día!\n")
    
    # Mostrar los viajes individuales de ese día
    detail_trips = df_audit[(df_audit['Ruta_ID'] == target_route) & (df_audit['Fecha'] == target_date)]
    
    print("Lista de envíos fragmentados (Evidencia):")
    display_cols = ['Viaje', 'F.Salida', 'Carrier_Name', 'Peso Total (kg)', 'Costo', 'TpoSrv']
    # Si alguna columna no existe, ajusta nombres
    valid_cols = [c for c in display_cols if c in detail_trips.columns]
    
    print(detail_trips[valid_cols].sort_values('F.Salida').to_string(index=False))
    
    # Cálculo de dinero tirado ese día específico
    costo_real = detail_trips['Costo'].sum()
    costo_teorico = (costo_real / worst_case['Viajes_Reales']) * worst_case['Camiones_Necesarios']
    dinero_tirado = costo_real - costo_teorico
    
    print(f"\n💸 Dinero desperdiciado solo en este día: ${dinero_tirado:,.2f}")

else:
    print("¡Felicidades! No se encontraron días con ineficiencias de consolidación obvias.")

# --- 3. VERIFICACIÓN DE CALIDAD DE DATOS (DATA HEALTH) ---
print("\n\n--- 3. VERIFICACIÓN DE CALIDAD DE DATOS ---")
print("Chequeo de valores extraños que podrían alterar el análisis:")
print(f"- Viajes con Peso 0 o negativo: {len(df_clean[df_clean['Peso Total (kg)'] <= 0])}")
print(f"- Viajes con Costo 0 o negativo: {len(df_clean[df_clean['Costo'] <= 0])}")
# Ver si hay outliers extremos de peso (> 60 toneladas, que sería ilegal o error)
print(f"- Viajes con Peso > 60 tons (Posible error de dedo): {len(df_clean[df_clean['Peso Total (kg)'] > 60000])}")